## Implementing Convolutional Neural Network from scratch

In [3]:
## Importing librairies
import numpy as np # type: ignore
from layer import Layer
from scipy import signal

ModuleNotFoundError: No module named 'layer'

In [ ]:
class CNN(Layer):
        
        def __init__(self, input_shape, kernel_size, depth):
                
                input_depth, input_height, input_width = input_shape
                self.input_shape = input_shape
                self.input_depth = input_depth
                self.input_height = input_height
                self.input_width = input_width
                self.kernel_size = kernel_size
                self.depth = depth # Represent the numbers of layers of kernel
                self.output_shape = (depth , self.input_height - self.kernel_size + 1, input_width - kernel_size + 1)
                self.kernel_shape = (depth, input_depth, kernel_size, kernel_size)
                
                ## Weight setup
                self.kernel = np.random.randn(self.kernel_shape)
                self.bias = np.random.randn(self.output_shape)

        def forward_propagation(self, input):
                
                self.input = input
                self.output = np.copy(self.bias)

                for i in range(self.depth):
                    for j in range(self.input_depth):    
                        self.output[i] += signal.correlate2d(self.input[i], self.kernel[i,j], 'valid')
                return self.output
        
        def background_propagation(self, output_gradient, learning_rate):
              
            kernels_gradient = np.zeros(self.kernel_shape)
            input_gradient = np.zeros(self.input_shape)

            for i in range(self.depth):
                for j in range(self.input_depth):
                    kernels_gradient[i][j] = signal.correlate2d(self.input[j], output_gradient[i], 'valid')
                    input_gradient[j] -= signal.convolve2d(output_gradient[i], self.kernel[i,j], 'full') 
            self.kernel -= learning_rate * kernels_gradient
            self.bias -= learning_rate * output_gradient

            return input_gradient

               


In [ ]:
class Reshape(Layer):

    def __init__(self, input_shape, output_shape):
        self.input_shape = input_shape
        self.output_shape = output_shape

    def forward_propagation(self, input):
        return np.reshape(input, self.output_shape)
    
    def backward_propagation(self, output_gradient, learning_rate):
        return np.reshape(output_gradient, self.input_shape)

In [ ]:
def cross_binary_entropy(y_true, y_pred):
    return - np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - np.log(1 - y_pred)))

def binary_cross_entropy_prime(y_true, y_pred):
    return ((1 - y_true) / (1 - y_pred) - y_true / y_pred ) / np.size(y_true)

In [ ]:
class Sigmoid:
    def __init__(self):
        def sigmoid(x):
            return 1 / (1 + np.exp( - x))
        def sigmoid_prime(x):
            s = sigmoid(x)
            return s * (1 - s)
        super.__init__(sigmoid, sigmoid_prime)